In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from sklearn.cluster import KMeans

rus= RandomUnderSampler(random_state= 15)
cc= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))

X_rus, y_rus= rus.fit_resample(X_train, y_train)
X_rus_cc, y_rus_cc= cc.fit_resample(X_rus, y_rus)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_rus_cc= LogisticRegression(random_state=15)
lr_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_lr_rus_cc= lr_rus_cc.predict(X_test)
y_pred_prob_lr_rus_cc= lr_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_rus_cc, labels= lr_rus_cc.classes_)
print(classification_report(y_test, y_pred_lr_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_lr_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84      8517
           1       0.25      0.81      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.77      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24757443961191034
Accuracy: 0.7432389436843779
AUC: 0.7736716968539701


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_rus_cc= DecisionTreeClassifier(random_state=15)
dt_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_dt_rus_cc= dt_rus_cc.predict(X_test)
y_pred_prob_dt_rus_cc= dt_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_rus_cc, labels= dt_rus_cc.classes_)
print(classification_report(y_test, y_pred_dt_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_rus_cc))

              precision    recall  f1-score   support

           0       0.95      0.65      0.77      8517
           1       0.17      0.66      0.27       912

    accuracy                           0.65      9429
   macro avg       0.56      0.66      0.52      9429
weighted avg       0.87      0.65      0.72      9429

Precision: 0.16887510572314632
Accuracy: 0.654152083996182
AUC: 0.6552327491559709


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_rus_cc= GridSearchCV(estimator= dt_rus_cc, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_rus_cc.fit(X_rus_cc, y_rus_cc)

print("Best Parameters:", gs_dt_rus_cc.best_params_)
print("Best Precision Score:", gs_dt_rus_cc.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10}
Best Precision Score: 0.7075413719123743


In [12]:
tuned_dt_rus_cc= gs_dt_rus_cc.best_estimator_
y_pred_tuned_dt_rus_cc= tuned_dt_rus_cc.predict(X_test)
y_pred_prob_tuned_dt_rus_cc= tuned_dt_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_rus_cc, labels= tuned_dt_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_dt_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_rus_cc))

              precision    recall  f1-score   support

           0       0.96      0.69      0.80      8517
           1       0.20      0.72      0.32       912

    accuracy                           0.70      9429
   macro avg       0.58      0.71      0.56      9429
weighted avg       0.89      0.70      0.76      9429

Precision: 0.201278928136419
Accuracy: 0.695195672923958
AUC: 0.7750207788756851


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_rus_cc= KNeighborsClassifier()
knn_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_knn_rus_cc= knn_rus_cc.predict(X_test)
y_pred_prob_knn_rus_cc= knn_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_rus_cc, labels= knn_rus_cc.classes_)
print(classification_report(y_test, y_pred_knn_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_rus_cc))

              precision    recall  f1-score   support

           0       0.96      0.70      0.81      8517
           1       0.21      0.72      0.32       912

    accuracy                           0.70      9429
   macro avg       0.58      0.71      0.57      9429
weighted avg       0.89      0.70      0.76      9429

Precision: 0.20586400499064256
Accuracy: 0.7032559126100328
AUC: 0.776320359796403


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_rus_cc= GridSearchCV(estimator= knn_rus_cc, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_rus_cc.fit(X_rus_cc, y_rus_cc)
print("Best Parameters:", gs_knn_rus_cc.best_params_)
print("Best Precision Score:", gs_knn_rus_cc.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.7440590692978164


In [15]:
tuned_knn_rus_cc= gs_knn_rus_cc.best_estimator_
y_pred_tuned_knn_rus_cc= tuned_knn_rus_cc.predict(X_test)
y_pred_prob_tuned_knn_rus_cc= tuned_knn_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_rus_cc, labels= tuned_knn_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_knn_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_rus_cc))

              precision    recall  f1-score   support

           0       0.94      0.83      0.88      8517
           1       0.23      0.47      0.31       912

    accuracy                           0.80      9429
   macro avg       0.58      0.65      0.60      9429
weighted avg       0.87      0.80      0.83      9429

Precision: 0.23164763458401305
Accuracy: 0.7986000636334712
AUC: 0.7275182606922378


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_rus_cc= GaussianNB()
nb_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_nb_rus_cc= nb_rus_cc.predict(X_test)
y_pred_prob_nb_rus_cc= nb_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_rus_cc, labels= nb_rus_cc.classes_)
print(classification_report(y_test, y_pred_nb_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_rus_cc))

              precision    recall  f1-score   support

           0       0.98      0.59      0.74      8517
           1       0.18      0.86      0.30       912

    accuracy                           0.62      9429
   macro avg       0.58      0.73      0.52      9429
weighted avg       0.90      0.62      0.69      9429

Precision: 0.1840074818798223
Accuracy: 0.616608335984728
AUC: 0.8059937915706257


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_rus_cc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_svc_rus_cc= svc_rus_cc.predict(X_test)
y_pred_prob_svc_rus_cc= svc_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_rus_cc, labels= svc_rus_cc.classes_)
print(classification_report(y_test, y_pred_svc_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_rus_cc))

              precision    recall  f1-score   support

           0       0.98      0.17      0.29      8517
           1       0.11      0.96      0.20       912

    accuracy                           0.25      9429
   macro avg       0.54      0.57      0.24      9429
weighted avg       0.89      0.25      0.28      9429

Precision: 0.11044025157232705
Accuracy: 0.24636758935199915
AUC: 0.6986473389650008


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_rus_cc= RandomForestClassifier(random_state=15)
rf_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_rf_rus_cc= rf_rus_cc.predict(X_test)
y_pred_prob_rf_rus_cc= rf_rus_cc.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_rus_cc, labels= rf_rus_cc.classes_)
print(classification_report(y_test, y_pred_rf_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8517
           1       0.23      0.83      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.77      0.59      9429
weighted avg       0.90      0.72      0.77      9429

Precision: 0.23126535626535627
Accuracy: 0.7176794994166932
AUC: 0.832226156562005


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_rus_cc= GridSearchCV(estimator= rf_rus_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_rus_cc.fit(X_rus_cc, y_rus_cc)
print("Best Parameters:", gs_tuned_rf_rus_cc.best_params_)
print("Best Precision Score:", gs_tuned_rf_rus_cc.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best Precision Score: 0.7297299712408059


In [21]:
tuned_rf_rus_cc= gs_tuned_rf_rus_cc.best_estimator_
y_pred_tuned_rf_rus_cc= tuned_rf_rus_cc.predict(X_test)
y_pred_prob_tuned_rf_rus_cc= tuned_rf_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_rus_cc, labels= tuned_rf_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_rf_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_rus_cc))

              precision    recall  f1-score   support

           0       0.98      0.71      0.82      8517
           1       0.23      0.83      0.37       912

    accuracy                           0.72      9429
   macro avg       0.60      0.77      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23469387755102042
Accuracy: 0.7212853961183583
AUC: 0.8470538283597923


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_rus_cc= AdaBoostClassifier(random_state=15)
ada_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_ada_rus_cc= ada_rus_cc.predict(X_test)
y_pred_prob_ada_rus_cc= ada_rus_cc.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_rus_cc, labels= ada_rus_cc.classes_)
print(classification_report(y_test, y_pred_ada_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25624133148404993
Accuracy: 0.7541626895747163
AUC: 0.8540979187136564


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_rus_cc= GridSearchCV(estimator= ada_rus_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_rus_cc.fit(X_rus_cc, y_rus_cc)
print("Best Parameters:", gs_ada_rus_cc.best_params_)
print("Best Precision Score:", gs_ada_rus_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 50}
Best Precision Score: 0.7428975812552493


In [25]:
tuned_ada_rus_cc= gs_ada_rus_cc.best_estimator_
y_pred_tuned_ada_rus_cc= tuned_ada_rus_cc.predict(X_test)
y_pred_prob_tuned_ada_rus_cc= tuned_ada_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_rus_cc, labels= tuned_ada_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_ada_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25624133148404993
Accuracy: 0.7541626895747163
AUC: 0.8540979187136564


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_rus_cc= GradientBoostingClassifier(random_state=15)
grb_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_grb_rus_cc= grb_rus_cc.predict(X_test)
y_pred_prob_grb_rus_cc= grb_rus_cc.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_rus_cc, labels= grb_rus_cc.classes_)
print(classification_report(y_test, y_pred_grb_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.73      0.84      8517
           1       0.25      0.82      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24842245101295252
Accuracy: 0.7426026089723194
AUC: 0.8512022652321775


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_rus_cc= GridSearchCV(estimator= grb_rus_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_rus_cc.fit(X_rus_cc, y_rus_cc)

print("Best Parameters:", gs_grb_rus_cc.best_params_)
print("Best Precision Score:", gs_grb_rus_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7352104819676042


In [28]:
tuned_grb_rus_cc= gs_grb_rus_cc.best_estimator_
y_pred_tuned_grb_rus_cc= tuned_grb_rus_cc.predict(X_test)
y_pred_prob_tuned_grb_rus_cc= tuned_grb_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_rus_cc, labels= tuned_grb_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_grb_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.73      0.84      8517
           1       0.25      0.82      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24842245101295252
Accuracy: 0.7426026089723194
AUC: 0.8512022652321775


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_rus_cc= XGBClassifier(random_state=15)
xgb_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_xgb_rus_cc= xgb_rus_cc.predict(X_test)
y_pred_prob_xgb_rus_cc= xgb_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_rus_cc, labels= xgb_rus_cc.classes_)
print(classification_report(y_test, y_pred_xgb_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_rus_cc))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8517
           1       0.23      0.79      0.35       912

    accuracy                           0.72      9429
   macro avg       0.60      0.75      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.22810913705583757
Accuracy: 0.7214975076890444
AUC: 0.8224834515695132


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_rus_cc= GridSearchCV(estimator= xgb_rus_cc, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_rus_cc.fit(X_rus_cc, y_rus_cc)

print("Best Parameters:", gs_xgb_rus_cc.best_params_)
print("Best Precision Score:", gs_xgb_rus_cc.best_score_)

tuned_xgb_rus_cc= gs_xgb_rus_cc.best_estimator_
y_pred_tuned_xgb_rus_cc= tuned_xgb_rus_cc.predict(X_test)
y_pred_prob_tuned_xgb_rus_cc= tuned_xgb_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_rus_cc, labels= tuned_xgb_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_rus_cc))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.726844666567301
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      8517
           1       0.24      0.81      0.37       912

    accuracy                           0.73      9429
   macro avg       0.60      0.77      0.60      9429
weighted avg       0.90      0.73      0.78      9429

Precision: 0.23630573248407644
Accuracy: 0.7276487432389437
AUC: 0.842692710554124


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_rus_cc= LGBMClassifier(random_state=15)
lgm_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_lgm_rus_cc= lgm_rus_cc.predict(X_test)
y_pred_prob_lgm_rus_cc= lgm_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_rus_cc, labels= lgm_rus_cc.classes_)
print(classification_report(y_test, y_pred_lgm_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_rus_cc))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.114537 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 225
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.97      0.72      0.82      8517
           1       0.23      0.81      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23419721871049304
Accuracy: 0.7248912928200233
AUC: 0.843141117146512


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_rus_cc= GridSearchCV(estimator= lgm_rus_cc, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_rus_cc.fit(X_rus_cc, y_rus_cc)

print("Best Parameters:", gs_lgm_rus_cc.best_params_)
print("Best Precision Score:", gs_lgm_rus_cc.best_score_)

tuned_lgm_rus_cc= gs_lgm_rus_cc.best_estimator_
y_pred_tuned_lgm_rus_cc= tuned_lgm_rus_cc.predict(X_test)
y_pred_prob_tuned_lgm_rus_cc= tuned_lgm_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_rus_cc, labels= tuned_lgm_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_rus_cc))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006804 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 198
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_rus_cc= CatBoostClassifier(random_state=15)
cat_rus_cc.fit(X_rus_cc, y_rus_cc)
y_pred_cat_rus_cc= cat_rus_cc.predict(X_test)
y_pred_prob_cat_rus_cc= cat_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_rus_cc, labels= cat_rus_cc.classes_)
print(classification_report(y_test, y_pred_cat_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_rus_cc))

Learning rate set to 0.024073
0:	learn: 0.6854674	total: 147ms	remaining: 2m 27s
1:	learn: 0.6779881	total: 154ms	remaining: 1m 16s
2:	learn: 0.6700761	total: 162ms	remaining: 53.7s
3:	learn: 0.6637142	total: 169ms	remaining: 42.1s
4:	learn: 0.6566018	total: 176ms	remaining: 35.1s
5:	learn: 0.6505401	total: 184ms	remaining: 30.5s
6:	learn: 0.6441643	total: 193ms	remaining: 27.3s
7:	learn: 0.6389221	total: 200ms	remaining: 24.8s
8:	learn: 0.6335225	total: 209ms	remaining: 23s
9:	learn: 0.6282914	total: 217ms	remaining: 21.5s
10:	learn: 0.6236142	total: 232ms	remaining: 20.8s
11:	learn: 0.6186825	total: 240ms	remaining: 19.8s
12:	learn: 0.6148313	total: 253ms	remaining: 19.2s
13:	learn: 0.6103098	total: 263ms	remaining: 18.5s
14:	learn: 0.6064800	total: 271ms	remaining: 17.8s
15:	learn: 0.6025335	total: 278ms	remaining: 17.1s
16:	learn: 0.5989408	total: 284ms	remaining: 16.4s
17:	learn: 0.5954332	total: 291ms	remaining: 15.9s
18:	learn: 0.5919803	total: 297ms	remaining: 15.4s
19:	learn: 

161:	learn: 0.4726954	total: 1.66s	remaining: 8.57s
162:	learn: 0.4724598	total: 1.66s	remaining: 8.55s
163:	learn: 0.4721725	total: 1.68s	remaining: 8.54s
164:	learn: 0.4719015	total: 1.68s	remaining: 8.53s
165:	learn: 0.4716789	total: 1.69s	remaining: 8.51s
166:	learn: 0.4714282	total: 1.7s	remaining: 8.5s
167:	learn: 0.4712651	total: 1.72s	remaining: 8.5s
168:	learn: 0.4710742	total: 1.73s	remaining: 8.48s
169:	learn: 0.4709311	total: 1.74s	remaining: 8.49s
170:	learn: 0.4707338	total: 1.75s	remaining: 8.49s
171:	learn: 0.4705154	total: 1.76s	remaining: 8.47s
172:	learn: 0.4703339	total: 1.77s	remaining: 8.46s
173:	learn: 0.4701619	total: 1.78s	remaining: 8.44s
174:	learn: 0.4698814	total: 1.79s	remaining: 8.42s
175:	learn: 0.4696433	total: 1.8s	remaining: 8.42s
176:	learn: 0.4694117	total: 1.81s	remaining: 8.41s
177:	learn: 0.4690874	total: 1.82s	remaining: 8.4s
178:	learn: 0.4689179	total: 1.83s	remaining: 8.39s
179:	learn: 0.4687309	total: 1.84s	remaining: 8.37s
180:	learn: 0.468

337:	learn: 0.4416792	total: 3.5s	remaining: 6.85s
338:	learn: 0.4415683	total: 3.5s	remaining: 6.83s
339:	learn: 0.4414231	total: 3.51s	remaining: 6.81s
340:	learn: 0.4412414	total: 3.52s	remaining: 6.8s
341:	learn: 0.4411074	total: 3.52s	remaining: 6.78s
342:	learn: 0.4409190	total: 3.53s	remaining: 6.77s
343:	learn: 0.4407074	total: 3.54s	remaining: 6.75s
344:	learn: 0.4406072	total: 3.55s	remaining: 6.73s
345:	learn: 0.4404336	total: 3.55s	remaining: 6.71s
346:	learn: 0.4402455	total: 3.56s	remaining: 6.7s
347:	learn: 0.4400829	total: 3.57s	remaining: 6.68s
348:	learn: 0.4398679	total: 3.58s	remaining: 6.68s
349:	learn: 0.4396154	total: 3.59s	remaining: 6.67s
350:	learn: 0.4395138	total: 3.61s	remaining: 6.67s
351:	learn: 0.4393133	total: 3.62s	remaining: 6.66s
352:	learn: 0.4391965	total: 3.63s	remaining: 6.66s
353:	learn: 0.4390485	total: 3.64s	remaining: 6.65s
354:	learn: 0.4389464	total: 3.66s	remaining: 6.65s
355:	learn: 0.4387868	total: 3.67s	remaining: 6.63s
356:	learn: 0.43

504:	learn: 0.4140297	total: 4.92s	remaining: 4.82s
505:	learn: 0.4138604	total: 4.92s	remaining: 4.81s
506:	learn: 0.4136631	total: 4.93s	remaining: 4.8s
507:	learn: 0.4135178	total: 4.94s	remaining: 4.79s
508:	learn: 0.4133700	total: 4.95s	remaining: 4.78s
509:	learn: 0.4132192	total: 4.96s	remaining: 4.76s
510:	learn: 0.4130381	total: 4.96s	remaining: 4.75s
511:	learn: 0.4129425	total: 4.97s	remaining: 4.74s
512:	learn: 0.4127322	total: 4.98s	remaining: 4.73s
513:	learn: 0.4125646	total: 4.99s	remaining: 4.72s
514:	learn: 0.4123542	total: 5s	remaining: 4.71s
515:	learn: 0.4121755	total: 5.01s	remaining: 4.7s
516:	learn: 0.4119976	total: 5.01s	remaining: 4.68s
517:	learn: 0.4118141	total: 5.03s	remaining: 4.67s
518:	learn: 0.4117413	total: 5.03s	remaining: 4.66s
519:	learn: 0.4115699	total: 5.04s	remaining: 4.65s
520:	learn: 0.4114349	total: 5.05s	remaining: 4.64s
521:	learn: 0.4112952	total: 5.06s	remaining: 4.63s
522:	learn: 0.4111002	total: 5.06s	remaining: 4.62s
523:	learn: 0.410

665:	learn: 0.3908281	total: 6.19s	remaining: 3.1s
666:	learn: 0.3907502	total: 6.2s	remaining: 3.09s
667:	learn: 0.3906329	total: 6.2s	remaining: 3.08s
668:	learn: 0.3905269	total: 6.21s	remaining: 3.07s
669:	learn: 0.3904897	total: 6.22s	remaining: 3.06s
670:	learn: 0.3903592	total: 6.23s	remaining: 3.05s
671:	learn: 0.3902973	total: 6.24s	remaining: 3.04s
672:	learn: 0.3901881	total: 6.25s	remaining: 3.04s
673:	learn: 0.3901209	total: 6.26s	remaining: 3.03s
674:	learn: 0.3900123	total: 6.27s	remaining: 3.02s
675:	learn: 0.3898382	total: 6.28s	remaining: 3.01s
676:	learn: 0.3896783	total: 6.28s	remaining: 3s
677:	learn: 0.3895779	total: 6.29s	remaining: 2.99s
678:	learn: 0.3893907	total: 6.3s	remaining: 2.98s
679:	learn: 0.3892091	total: 6.3s	remaining: 2.97s
680:	learn: 0.3891106	total: 6.31s	remaining: 2.96s
681:	learn: 0.3889659	total: 6.32s	remaining: 2.95s
682:	learn: 0.3888263	total: 6.33s	remaining: 2.94s
683:	learn: 0.3886494	total: 6.33s	remaining: 2.92s
684:	learn: 0.388622

832:	learn: 0.3709182	total: 7.47s	remaining: 1.5s
833:	learn: 0.3708014	total: 7.48s	remaining: 1.49s
834:	learn: 0.3706792	total: 7.49s	remaining: 1.48s
835:	learn: 0.3705750	total: 7.49s	remaining: 1.47s
836:	learn: 0.3704609	total: 7.5s	remaining: 1.46s
837:	learn: 0.3703534	total: 7.51s	remaining: 1.45s
838:	learn: 0.3701955	total: 7.52s	remaining: 1.44s
839:	learn: 0.3701374	total: 7.53s	remaining: 1.43s
840:	learn: 0.3700094	total: 7.53s	remaining: 1.42s
841:	learn: 0.3698918	total: 7.54s	remaining: 1.42s
842:	learn: 0.3698081	total: 7.55s	remaining: 1.41s
843:	learn: 0.3696726	total: 7.56s	remaining: 1.4s
844:	learn: 0.3695228	total: 7.56s	remaining: 1.39s
845:	learn: 0.3693989	total: 7.57s	remaining: 1.38s
846:	learn: 0.3692844	total: 7.58s	remaining: 1.37s
847:	learn: 0.3691566	total: 7.58s	remaining: 1.36s
848:	learn: 0.3690368	total: 7.59s	remaining: 1.35s
849:	learn: 0.3689406	total: 7.6s	remaining: 1.34s
850:	learn: 0.3688542	total: 7.61s	remaining: 1.33s
851:	learn: 0.36

999:	learn: 0.3533372	total: 8.75s	remaining: 0us
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      8517
           1       0.24      0.82      0.37       912

    accuracy                           0.73      9429
   macro avg       0.61      0.77      0.60      9429
weighted avg       0.90      0.73      0.78      9429

Precision: 0.24024312220089572
Accuracy: 0.7310425283699226
AUC: 0.8458540864607216


In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_rus_cc= GridSearchCV(estimator= cat_rus_cc, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_rus_cc.fit(X_rus_cc, y_rus_cc)

print("Best Parameters:", gs_cat_rus_cc.best_params_)
print("Best Precision Score:", gs_cat_rus_cc.best_score_)

tuned_cat_rus_cc= gs_cat_rus_cc.best_estimator_
y_pred_tuned_cat_rus_cc= tuned_cat_rus_cc.predict(X_test)
y_pred_prob_tuned_cat_rus_cc= tuned_cat_rus_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_rus_cc, labels= tuned_cat_rus_cc.classes_)
print(classification_report(y_test, y_pred_tuned_cat_rus_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_rus_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_rus_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_rus_cc))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6705162	total: 3.4ms	remaining: 337ms
1:	learn: 0.6505512	total: 7.08ms	remaining: 347ms
2:	learn: 0.6344977	total: 11.2ms	remaining: 361ms
3:	learn: 0.6205024	total: 15.4ms	remaining: 370ms
4:	learn: 0.6085720	total: 20.3ms	remaining: 385ms
5:	learn: 0.5996192	total: 26.4ms	remaining: 414ms
6:	learn: 0.5910311	total: 32.7ms	remaining: 434ms
7:	learn: 0.5838536	total: 38.2ms	remaining: 439ms
8:	learn: 0.5763569	total: 44ms	remaining: 445ms
9:	learn: 0.5698887	total: 49.5ms	remaining: 446ms
10:	learn: 0.5645359	total: 54.8ms	remaining: 443ms
11:	learn: 0.5603481	total: 60.1ms	remaining: 441ms
12:	learn: 0.5567605	total: 70.1ms	remaining: 469ms
13:	learn: 0.5523547	total: 78.1ms	remaining: 480ms
14:	learn: 0.5479060	total: 84.3ms	remaining: 478ms
15:	learn: 0.5444808	total: 92.6ms	remaining: 486ms
16:	learn: 0.5415266	total: 101ms	remaining: 492ms
17:	learn: 0.5385793	total: 109ms	remaining: 497ms
18:	learn: 0.5361

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_rus_cc, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_rus_cc, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_rus_cc, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_rus_cc, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_rus_cc, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_rus_cc, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_rus_cc, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_rus_cc, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_rus_cc, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_rus_cc, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_rus_cc, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_rus_cc, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_rus_cc, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_rus_cc, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_rus_cc, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_rus_cc, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_rus_cc, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_rus_cc, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_rus_cc, zero_division= 0))

Logistic Regression Precision: 0.24757443961191034
Decision Tree Precision: 0.16887510572314632
Tuned Decision Tree Precision: 0.201278928136419
KNeighborsClassifier Precision: 0.20586400499064256
Tuned KNeighborsClassifier Precision: 0.23164763458401305
GaussianNB Precision: 0.1840074818798223
SVM Precision: 0.11044025157232705
Random Forest Precision: 0.23126535626535627
Tuned Random Forest Precision: 0.23469387755102042
AdaBoost Precision: 0.25624133148404993
Tuned AdaBoost Precision: 0.25624133148404993
GradientBoosting Precision: 0.24842245101295252
Tuned GradientBoosting Precision: 0.24842245101295252
XGB Precision: 0.22810913705583757
Tuned XGB Precision: 0.23630573248407644
LGBM Precision: 0.23419721871049304
Tuned LGBM Precision: 0.24311338885329917
CatBoost Precision: 0.24024312220089572
Tuned CatBoost Precision: 0.2509894459102902


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_rus_cc, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_rus_cc, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_rus_cc, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_rus_cc, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_rus_cc, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_rus_cc, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_rus_cc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_rus_cc, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_rus_cc, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_rus_cc, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_rus_cc, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_rus_cc, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_rus_cc, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_rus_cc, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_rus_cc, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_rus_cc, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_rus_cc, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_rus_cc, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_rus_cc, zero_division= 0)
                  }

rus_cc_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rus_cc_precision= rus_cc_precision.sort_values(by= 'Precision Score', ascending=False)
print(rus_cc_precision)

                                    Model  Precision Score
9                     AdaBoost Precision:         0.256241
10              Tuned AdaBoost Precision:         0.256241
18              Tuned CatBoost Precision:         0.250989
11            GradientBoosting Precision:         0.248422
12      Tuned GradientBoosting Precision:         0.248422
0          Logistic Regression Precision:         0.247574
16                  Tuned LGBM Precision:         0.243113
17                    CatBoost Precision:         0.240243
14                   Tuned XGB Precision:         0.236306
8          Tuned Random Forest Precision:         0.234694
15                        LGBM Precision:         0.234197
4   Tuned KNeighborsClassifier Precision:         0.231648
7                Random Forest Precision:         0.231265
13                         XGB Precision:         0.228109
3         KNeighborsClassifier Precision:         0.205864
2          Tuned Decision Tree Precision:         0.2012

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_rus_cc),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_rus_cc),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_rus_cc),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_rus_cc),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_rus_cc),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_rus_cc),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_rus_cc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_rus_cc),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_rus_cc),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_rus_cc),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_rus_cc),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_rus_cc),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_rus_cc),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_rus_cc),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_rus_cc),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_rus_cc),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_rus_cc),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_rus_cc),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_rus_cc)
                  }

rus_cc_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rus_cc_accuracy= rus_cc_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rus_cc_accuracy)

                                   Model  Accuracy Score
4   Tuned KNeighborsClassifier Accuracy:        0.798600
9                     AdaBoost Accuracy:        0.754163
10              Tuned AdaBoost Accuracy:        0.754163
0          Logistic Regression Accuracy:        0.743239
18              Tuned CatBoost Accuracy:        0.743133
11            GradientBoosting Accuracy:        0.742603
12      Tuned GradientBoosting Accuracy:        0.742603
16                  Tuned LGBM Accuracy:        0.733164
17                    CatBoost Accuracy:        0.731043
14                   Tuned XGB Accuracy:        0.727649
15                        LGBM Accuracy:        0.724891
13                         XGB Accuracy:        0.721498
8          Tuned Random Forest Accuracy:        0.721285
7                Random Forest Accuracy:        0.717679
3         KNeighborsClassifier Accuracy:        0.703256
2          Tuned Decision Tree Accuracy:        0.695196
1                Decision Tree 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_rus_cc),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_rus_cc),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_rus_cc),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_rus_cc),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_rus_cc),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_rus_cc),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_rus_cc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_rus_cc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_rus_cc),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_rus_cc),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_rus_cc),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_rus_cc),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_rus_cc),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_rus_cc),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_rus_cc),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_rus_cc),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_rus_cc),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_rus_cc),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_rus_cc)
                  }

rus_cc_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rus_cc_auc= rus_cc_auc.sort_values(by= 'AUC Score', ascending=False)
print(rus_cc_auc)

                              Model  AUC Score
18              Tuned CatBoost AUC:   0.854178
10              Tuned AdaBoost AUC:   0.854098
9                     AdaBoost AUC:   0.854098
0          Logistic Regression AUC:   0.853814
11            GradientBoosting AUC:   0.851202
12      Tuned GradientBoosting AUC:   0.851202
16                  Tuned LGBM AUC:   0.849928
8          Tuned Random Forest AUC:   0.847054
17                    CatBoost AUC:   0.845854
15                        LGBM AUC:   0.843141
14                   Tuned XGB AUC:   0.842693
7                Random Forest AUC:   0.832226
13                         XGB AUC:   0.822483
5                   GaussianNB AUC:   0.805994
3         KNeighborsClassifier AUC:   0.776320
2          Tuned Decision Tree AUC:   0.775021
4   Tuned KNeighborsClassifier AUC:   0.727518
6                          SVM AUC:   0.698647
1                Decision Tree AUC:   0.655233
